In [1]:
import pandas as pd
import matplotlib.pyplot as plt
import matplotlib as mpl
import os
import glob
import numpy as np
import statistics
import math
from shutil import copyfile
from statsmodels.graphics.gofplots import qqplot
from scipy.stats import shapiro
import scipy
from collections import Counter


In [2]:
position = ["position_x","position_y","position_z"]
target= ["target_position_x","target_position_y","target_position_z"]
total_time = "time"

max_distance_error = 4.0

max_height_error = 3*max_distance_error/4
max_contraction_error = 1.0
feedback_types = ['Visual', 'Glove', 'Bracelets']
question_dict = {'Q1': 'How mentally demanding was the test ?', 'Q2': 'How hurried or rushed was the pace of the task?', 'Q3': 'How physically demanding was the test', 'Q4': 'How successful were you in accomplishing the task ?', 'Q5': 'How insecure, discouraged, irritated, stressed and annoyed were you ?'}
information_types = ["height", "directional","extension",  'task in general']

In [14]:
# plot settings

lw = 1.5

fs = 13

params = {
    'axes.labelsize': fs,
    'font.size': fs,
    'legend.fontsize': fs,
    'xtick.labelsize': fs,
    'ytick.labelsize': fs,
    #'text.usetex' : True,
    'figure.figsize': [4, 4],
    'boxplot.boxprops.linewidth' : lw,
    'boxplot.whiskerprops.linewidth' : lw,
    'boxplot.capprops.linewidth' : lw,
    'boxplot.medianprops.linewidth' : lw,
    'font.family' : 'serif'
    

   }
mpl.rcParams.update(params)

In [15]:
os.chdir('..\Bidirectional_interface\Assets\Logs')

FileNotFoundError: [WinError 3] Le chemin d’accès spécifié est introuvable: '..\\Bidirectional_interface\\Assets\\Logs'

In [16]:
os.mkdir("survey_results")


FileExistsError: [WinError 183] Impossible de créer un fichier déjà existant: 'survey_results'

In [17]:
files = glob.glob("survey_results//*.csv")

In [18]:
#visual_df = pd.read_csv(folder + "NASA_visual_results.csv")
def attribute_files() :
    global visual_df, bracelets_df, glove_df,general_df
    for file in files :
        print(file)
        if "visual" in file:
            visual_df = pd.read_csv(file)
        elif "glove" in file:
            glove_df = pd.read_csv(file)
        elif "bracelets" in file:
            bracelets_df = pd.read_csv(file)
        elif "general" in file:
            general_df = pd.read_csv(file)

In [19]:
attribute_files()

survey_results\general_survey_modif.csv
survey_results\NASA_bracelets_results.csv
survey_results\NASA_glove_results.csv
survey_results\NASA_visual_results.csv


In [20]:
column_name = list(general_df.columns)
column_name.pop(0)
column_name

['Subject n',
 'Age',
 'Gender',
 'Which feedback system was easier to use for height (up/down) control?',
 'Which feedback system did you prefer to use for height (up/down) control?',
 'How useful were the bracelets for height (up/down) control??',
 'How useful was the glove for height (up/down) control??',
 'How useful was the visual feedback for height (up/down) control??',
 'Which feedback system was easier to use for planar directional (front/back/left/right) control ?',
 'Which feedback system did you prefer to use for planar directional (front/back/left/right) control ?',
 'How useful were the bracelets for horizontal directional (front/back/left/right) control ?',
 'How useful was the glove for horizontal directional (front/back/left/right) control ?',
 'How useful was the visual feedback for horizontal directional (front/back/left/right) control ?',
 'Which feedback system was easier to use for extension/contraction control ?',
 'Which feedback system did you prefer to use for

In [21]:
def plot_boxplot_NASA(Question):
    # Define styling for each boxplot component
    medianprops = {'color': 'magenta', 'linewidth': 2}
    boxprops = {'color': 'black', 'linestyle': '-', 'linewidth':2}
    whiskerprops = {'color': 'black', 'linestyle': '-'}
    capprops = {'color': 'black', 'linestyle': '-'}
    flierprops = {'color': 'black', 'marker': 'x'}
 
    fig = plt.figure()
    ax = plt.axes()

    width = 0.5
    bp = plt.boxplot([visual_df[Question], glove_df[Question], bracelets_df[Question]],
                     widths = width, labels = feedback_types,medianprops=medianprops,
                       boxprops=boxprops,
                       whiskerprops=whiskerprops,
                     patch_artist=True,  # fill with color
                       capprops=capprops,
                       flierprops=flierprops )

    colors = ['pink', 'lightblue', 'lightgreen']
    for patch, color in zip(bp['boxes'], colors):
        patch.set_facecolor(color)
        
    
    plt.title(question_dict[Question])
    ax.set_xlabel('Feedback type')
    ax.set_ylabel('Time [s]')
    ax.set_ylim(-10,110)
        

In [22]:
for question in question_dict:
    plot_boxplot_NASA(question)

RuntimeError: Failed to process string with tex because latex could not be found

<Figure size 288x288 with 1 Axes>

RuntimeError: Failed to process string with tex because latex could not be found

<Figure size 288x288 with 1 Axes>

RuntimeError: Failed to process string with tex because latex could not be found

<Figure size 288x288 with 1 Axes>

RuntimeError: Failed to process string with tex because latex could not be found

<Figure size 288x288 with 1 Axes>

RuntimeError: Failed to process string with tex because latex could not be found

<Figure size 288x288 with 1 Axes>

In [ ]:
ages = np.sort(general_df['Age'])
print("The youngest subject was " + str(ages[0]) + " and the oldest was " + str(ages[-1]))

# Pie charts

In this section are plotted the pie showing which feedback system the user prefered and which one they found easier to use

In [ ]:
def find_column_name(type_of_info):
    column_names = {}
    for title in list(general_df.columns):
        if type_of_info in title and ("system" in title or "interface" in title):
            if "prefer" in title:
                column_names["prefer"] = title
            elif "easier" in title:
                column_names["easier"] = title
                
    return column_names

In [ ]:
def plot_pies(type_of_information):
    
    plt.figure(figsize = (12,12))
    column_names = find_column_name(type_of_information)

    easier_pie_data = Counter(general_df[column_names["easier"]])
    prefered_pie_data = Counter(general_df[column_names["prefer"]])

#     ax1 = plt.subplot(221)
#     ax1.pie(gender_pie_data.values(), labels=gender_pie_data.keys(), autopct='%1.1f%%', startangle=90)
#     ax1.axis('equal')  # Equal aspect ratio ensures that pie is drawn as a circle.
#     ax1.set_title(gender)

    ax1 = plt.subplot(231)
    ax1.pie(easier_pie_data.values(), labels=easier_pie_data.keys(), autopct='%1.1f%%', startangle=90)
    ax1.axis('equal')  # Equal aspect ratio ensures that pie is drawn as a circle.
    ax1.set_title(column_names['easier'])


    ax1 = plt.subplot(233)
    ax1.pie(prefered_pie_data.values(), labels=prefered_pie_data.keys(), autopct='%1.1f%%', startangle=90)
    ax1.axis('equal')  # Equal aspect ratio ensures that pie is drawn as a circle.
    ax1.set_title(column_names['prefer'])

    plt.show()
    

In [ ]:
for info_type in information_types:
    plot_pies(info_type)

# Boxplot usefulleness

In this section are plotted "How useful was this device for this information"

In [ ]:
def find_column_name_bxplot(type_of_info):
    column_names = {}
    for title in list(general_df.columns):
        if type_of_info in title and "useful" in title:
            if "visual" in title:
                column_names["Visual"] = title
            elif "glove" in title:
                column_names["Glove"] = title
            elif "bracelets" in title:
                column_names["Bracelets"] = title
           
                
    return column_names

In [ ]:
find_column_name_bxplot("directional")

In [ ]:
column_names = find_column_name_bxplot("directional")
[column_names[feedback_types[0]], column_names[feedback_types[1]], column_names[feedback_types[2]]]

In [ ]:
def boxplot_useful(information_type):
    # Define styling for each boxplot component
    medianprops = {'color': 'magenta', 'linewidth': 2}
    boxprops = {'color': 'black', 'linestyle': '-', 'linewidth':2}
    whiskerprops = {'color': 'black', 'linestyle': '-'}
    capprops = {'color': 'black', 'linestyle': '-'}
    flierprops = {'color': 'black', 'marker': 'x'}
 
    fig = plt.figure()
    ax = plt.axes()

    width = 0.5
    column_names = find_column_name_bxplot(information_type)
    bp = plt.boxplot([10*general_df[column_names[feedback_types[0]]], 10*general_df[column_names[feedback_types[1]]], 10*general_df[column_names[feedback_types[2]]]],
                     widths = width, labels = feedback_types,medianprops=medianprops,
                       boxprops=boxprops,
                       whiskerprops=whiskerprops,
                     patch_artist=True,  # fill with color
                       capprops=capprops,
                       flierprops=flierprops )

    colors = ['pink', 'lightblue', 'lightgreen']
    for patch, color in zip(bp['boxes'], colors):
        patch.set_facecolor(color)
        
    
    plt.title(column_names[feedback_types[0]].replace("visual ", ""))
    ax.set_xlabel('Feedback type')
    ax.set_ylabel('Time [s]')
    ax.set_ylim(-10,110)

In [ ]:
for info_type in information_types:
    boxplot_useful(info_type)